 <a name="1"></a>
# <span style="color:green">Задачи исследования:<span>
    
- проанализировать тарифные планы компании
- проанализировать поведение клиентов компании использующие тарифные планы представленные ниже
- сделать вывод — какой тариф приносит больше денег.

### Описание тарифов

##### Тариф «Смарт»

1. Ежемесячная плата: 550 рублей
2. Включено 500 минут разговора, 50 сообщений и 15 Гб интернет-трафика
3. Стоимость услуг сверх тарифного пакета:
    - минута разговора: 3 рубля («Мегалайн» всегда округляет вверх значения минут и мегабайтов.
    Если пользователь проговорил всего 1 секунду, в тарифе засчитывается целая минута.)
    - сообщение: 3 рубля
    - 1 Гб интернет-трафика: 200 рублей

##### Тариф «Ультра»

1. Ежемесячная плата: 1950 рублей
2. Включено 3000 минут разговора, 1000 сообщений и 30 Гб интернет-трафика
3. Стоимость услуг сверх тарифного пакета:
    - минута разговора: 1 рубль
    - сообщение: 1 рубль
    - 1 Гб интернет-трафика: 150 рублей

# План 

## 1. [Осмотр и изучение данных](#1)

## 2. [Предобработка данных](#2)

- исправление ошибок в данных
- проверка и предварительный анализ

## 3. [Подготовка, очистка и добавление данных](#3)

- индексы и названия столбцов
- работа с дубликатами
- работа с пропущенными значениями
- изменение типов данных
- добавление дополнительных метрик **для каждого пользователя:**
      
      
    - количество сделанных звонков и израсходованных минут разговора по месяцам;
    - количество отправленных сообщений по месяцам;
    - объем израсходованного интернет-трафика по месяцам;
    - помесячную выручку с каждого пользователя (вычтите бесплатный лимит из суммарного количества звонков, сообщений и интернет-трафика; остаток умножьте на значение из тарифного плана).

## 4. [Исследовательский анализ данных](#4)

- Опишите поведение клиентов оператора, исходя из выборки.
- Сколько минут разговора, сколько сообщений и какой объём интернет-трафика требуется пользователям каждого тарифа в месяц?
- Посчитайте среднее количество, дисперсию и стандартное отклонение. Постройте гистограммы. Опишите распределения.(#41)
  
## 5. [Проверка гипотез](#5)

- средняя выручка пользователей тарифов «Ультра» и «Смарт» различается;
- средняя выручка пользователей из Москвы отличается от выручки пользователей из других регионов;
  
## 6. [Выводы и результаты исследования](#6)

# 1. Импорт библиотек и чтение данных

In [2]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Disable warnings in Anaconda
import warnings
warnings.filterwarnings('ignore')

from pylab import rcParams
rcParams['figure.figsize'] = 10,7
pd.set_option('display.max_columns', None)

In [4]:
calls = pd.read_csv('datasets/calls.csv')
internet = pd.read_csv('datasets/internet.csv')
messages = pd.read_csv('datasets/messages.csv')
tariffs = pd.read_csv('datasets/tariffs.csv')
users = pd.read_csv('datasets/users.csv')

In [9]:
calls.head(3)

,id,call_date,duration,user_id
0,1000_0,2018-07-25,0.00,1000
1,1000_1,2018-08-17,0.00,1000
2,1000_2,2018-06-11,2.85,1000


In [10]:
internet.head(3)

,id,mb_used,session_date,user_id
0,1000_0,112.95,2018-11-25,1000
1,1000_1,1052.81,2018-09-07,1000
2,1000_2,1197.26,2018-06-25,1000


In [11]:
messages.head(3)

,id,message_date,user_id
0,1000_0,2018-06-27,1000
1,1000_1,2018-10-08,1000
2,1000_2,2018-08-04,1000


In [12]:
users.head(3)

,user_id,age,churn_date,city,first_name,last_name,reg_date,tariff
0,1000,52,NaN,Краснодар,Рафаил,Верещагин,2018-05-25,ultra
1,1001,41,NaN,Москва,Иван,Ежов,2018-11-01,smart
2,1002,59,NaN,Стерлитамак,Евгений,Абрамович,2018-06-17,smart


In [14]:
tariffs.head()

,messages_included,mg_per_month_included,minutes_included,rub_monthly_fee,rub_per_gb,rub_per_message,rub_per_minute,tariff_name
0,50,15360,500,550,200,3,3,smart
1,1000,30720,3000,1950,150,1,1,ultra


In [16]:
len(users), len(calls), len(messages), len(internet)

(500, 202607, 123036, 149396)

In [27]:
all_calls_user = calls.pivot_table(index='user_id', values='duration')
all_calls_user.head()

,duration
user_id,
1000,6.571304
1001,6.552951
1002,6.595253
1003,6.924077
1004,5.234148


In [30]:
all_internet_user = internet.pivot_table(index='user_id', values='mb_used')
all_internet_user.head()

,mb_used
user_id,
1000,508.968294
1001,338.187500
1002,314.462114
1003,550.775385
1004,496.798267


In [43]:
all_messages_user = messages.pivot_table(index='user_id', aggfunc='count')
all_messages_user.drop(['id'], axis=1, inplace=True)
all_messages_user.rename({'message_date': 'count_messages'}, axis=1, inplace=True)
all_messages_user.head()

,count_messages
user_id,
1000,496
1002,70
1003,380
1004,1192
1005,552


In [ ]:
## Смержим общие данные для представления общей картины

In [47]:
df = ( 
        users.merge(all_calls_user, how='outer', on='user_id')
             .merge(all_internet_user, how='outer', on='user_id')
             .merge(all_messages_user, how='outer', on='user_id')
      )

In [48]:
df.head()

,user_id,age,churn_date,city,first_name,last_name,reg_date,tariff,duration,mb_used,count_messages
0,1000,52,NaN,Краснодар,Рафаил,Верещагин,2018-05-25,ultra,6.571304,508.968294,496.0
1,1001,41,NaN,Москва,Иван,Ежов,2018-11-01,smart,6.552951,338.187500,NaN
2,1002,59,NaN,Стерлитамак,Евгений,Абрамович,2018-06-17,smart,6.595253,314.462114,70.0
3,1003,23,NaN,Москва,Белла,Белякова,2018-08-17,ultra,6.924077,550.775385,380.0
4,1004,68,NaN,Новокузнецк,Татьяна,Авдеенко,2018-05-14,ultra,5.234148,496.798267,1192.0


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 0 to 499
Data columns (total 11 columns):
user_id           500 non-null int64
age               500 non-null int64
churn_date        38 non-null object
city              500 non-null object
first_name        500 non-null object
last_name         500 non-null object
reg_date          500 non-null object
tariff            500 non-null object
duration          492 non-null float64
mb_used           497 non-null float64
count_messages    426 non-null float64
dtypes: float64(3), int64(2), object(6)
memory usage: 46.9+ KB


 <a name="2"></a>
# 2. Предобработка данных и предварительный анализ
[К оглавлению](#1)

<a name="3"></a>
# 3. Подготовка и очистка данных
[К оглавлению](#1)